In [ ]:
import nibabel as nib
import matplotlib.pyplot as plt
import os
import numpy as np
import queue
import pandas as pd

from scipy.ndimage import morphology

In [4]:
def surfd_min(input1, input2, sampling=1, connectivity=1): #3D, https://mlnotebook.github.io/post/surface-distance-function/
    #2D or 3D minimal intercarotid distance?
    
    input_1 = np.atleast_1d(input1.astype(bool))
    input_2 = np.atleast_1d(input2.astype(bool))
    
    conn = morphology.generate_binary_structure(input_1.ndim, connectivity)

    S = input_1 ^ morphology.binary_erosion(input_1, conn)
    Sprime = input_2 ^ morphology.binary_erosion(input_2, conn)

    dta = morphology.distance_transform_edt(~S,sampling)
    dtb = morphology.distance_transform_edt(~Sprime,sampling)
    
    sds = np.concatenate([np.ravel(dta[Sprime!=0]), np.ravel(dtb[S!=0])])
       
    return np.min(sds)

def zps_distances(mask_path):
    tumor = nib.load(mask_path).get_fdata()
    tumor[tumor != 1] = 0

    carotid_left = nib.load(mask_path).get_fdata()
    carotid_left[carotid_left !=3] = 0
    carotid_left[carotid_left ==3] = 1

    carotid_right = nib.load(mask_path).get_fdata()
    carotid_right[carotid_right !=2] = 0
    carotid_right[carotid_right ==2] = 1
    
    max_slice = []
    for y in range(256):
        tumor_slice = tumor[:,y,:]
        #plt.imshow(tumor_slice, cmap='gray')
        #plt.show()
        horizontal_size = []
        for i in range(256):
            tumor_line = tumor_slice[:,i]
            horizontal_size.append(np.sum(tumor_line))
        max_slice.append(np.max(horizontal_size))
        
    a = np.max(max_slice)
    print("the maximum is on slice", np.argmax(max_slice))
    b = surfd_min(carotid_right, carotid_left)
    c = a/b
    print(mask_path)
    print("ratio:",c, "intercarotid distance:", b, "max tumor:", a)

    return (c,a,b)

def my_add(xs, ys):
    return (xs[0] + ys[0],xs[1]+ys[1])

def good_tuple(xs):
    x = xs[0]
    y = xs[1]
    if x >= 0 and x < 256 and y >= 0 and y < 256:
        return True
    return False
## Tuple add, this should exist somewhere as a lib function, but ill write it anyway
def my_add(xs, ys):
    return (xs[0] + ys[0],xs[1]+ys[1])

def good_tuple(xs):
    x = xs[0]
    y = xs[1]
    if x >= 0 and x < 256 and y >= 0 and y < 256:
        return True
    return False


## This function outputs true if the tumor encloses the artery completely on this slice!
neighbours = [(1,0),(0,1),(-1,0),(0,-1),(1,1),(1,-1),(-1,-1),(-1,1)]
def check_Tumor(arterie_num, slice):
    Flag = False ## Check Flag if Tumor exists on this slice
    ##slice[0,0] = arterie_num
    q = queue.Queue() ## create queue for bfs
    ## put every artery bit onto the queue!
    for i in range(0,slice.shape[0]):
        for j in range(0,slice.shape[1]):
            if slice[i,j] == arterie_num:
                q.put((i,j))
            if slice[i,j] == 1:
                Flag = True
    ## If Flag = False here, then there is no tumor on this slice therefore it can't enclose the artery!
    if not Flag:
        return (False,False)
    ## Do BFS to "grow" artery
    while q.qsize() > 0:
        current_idx = q.get()
        ## now check all neighbours
        for nex in neighbours:
            neigh = my_add(current_idx,nex)
            if good_tuple(neigh) and slice[neigh] == 0:
                slice[neigh] = arterie_num
                q.put(neigh)
    ## We now check whether there exists a way for the grown artery to touch the border
    ## If that were the case, then the tumor does not enclose the artery on that slice
    N = 256
    for idx in range(0,N):
        if slice[0,idx] == arterie_num or slice[N-1,idx] == arterie_num or slice[idx,0] == arterie_num or slice[idx,N-1] == arterie_num:
                return (False,Flag)
    return (True,Flag)

def ZPS (mask_path):
    zps = 0
    tumor = nib.load(mask_path).get_fdata()

    for y in range(256):

        enclosed_right = 0
        enclosed_left = 0
        tumor_slice = tumor[:,y,:]
        if np.sum(tumor_slice) == 0: 
            continue
        if 2 in tumor_slice and 3 in tumor_slice:
            #print("analyzing:",y)
            enclosed_right = check_Tumor(2, tumor_slice.copy()) ## check right arterie
            enclosed_left = check_Tumor(3, tumor_slice.copy()) ## check left arterie
            if enclosed_right[0] and enclosed_right[1]:
                print("on Slice: " + str(y) + " the tumor encloses the right artery completely!")
                plt.imshow(tumor_slice, cmap='gray')
                plt.show()
                zps = 4
            if enclosed_left[0] and enclosed_left[1]:
                print("on Slice: " + str(y) + " the tumor encloses the left artery completely!")
                plt.imshow(tumor_slice, cmap='gray')
                plt.show()
                zps=4
        y = y+1

    if zps == 4:
        zps = 4
    
    if zps == 0:
        c = zps_distances(mask_path)
        c = c[0]
        if c < 0.75: 
            zps = 1
        elif (c > 0.75) & (c < 1.25):
            zps = 2
        elif c > 1.25:
            zps = 3
    return zps

In [ ]:
path_dir = ('') #path to directory with segmentations in NiFTI format
dir_cont = os.listdir(path_dir)
dir_cont.sort()
print(dir_cont)
zps_list = []
name = []
max_tumor_list = []
intercarotid_list = []
ratio = []

a = 1

for i in dir_cont:
    if i == ".DS_Store":
        continue
    else:
        try:
            y = os.path.join(path_dir, i)

            x = ZPS(y)
            c = zps_distances(y)
            d = c[0]
            e = c[1]# max horizontal tumor
            f = c[2]#ntercarotid distance

            zps_list.append(x)
            name.append(i)
            ratio.append(d)
            max_tumor_list.append(e)
            intercarotid_list.append(f)
            
            print(i,"-->", x)
            a= a+1
        except ValueError:
            print(i, "has a ValueError")
            zps_list.append(np.nan)
            name.append(np.nan)
            max_tumor_list.append(np.nan)
            ratio.append(np.nan)
            intercarotid_list.append(np.nan)
print(zps_list)


In [ ]:
df = pd.DataFrame()
df['name']= name
df['predicted_zps']= zps_list
df['minimal intercarotid distance'] = intercarotid_list
df['max_tumor_list'] = max_tumor_list
df['ratio'] = ratio
print(df)
path_save = ''
df.to_csv(path_save +'results.xls')